# NER Tagging by CRF Suite

The dataset we will be using is  GMB(Groningen Meaning Bank) corpus for entity classification. I have downloaded it from Kaggle for demonstration.

## Step 1 : Importing Data
The data is in json format and has data in format.
* Each element in parsed is a sentence
* Each element in the sentence sublist is primary word feature containing *\[word, pos_tag, ner_tag(label) \]*

In [54]:
import json
import pprint
with open("data.json", 'rb') as json_data:
    json_data_content = list(json_data)
    json_data_content = json_data_content[0] 
    doc = json.loads(json_data_content)

### Example of parsing elements from dataset

In [46]:
# How to parse words from sentences 
for sentence_idx,sentence in enumerate(doc):
    print ('sentence_idx : ',sentence_idx)
    for word_idx,word in enumerate(sentence):
        print ('word : ',word)
    if i > 5 :break;


sentence_idx :  0
word :  [['Thousands', 'NNS'], 'O']
word :  [['of', 'IN'], 'O']
word :  [['demonstrators', 'NNS'], 'O']
word :  [['have', 'VBP'], 'O']
word :  [['marched', 'VBN'], 'O']
word :  [['through', 'IN'], 'O']
word :  [['London', 'NNP'], 'B-geo']
word :  [['to', 'TO'], 'O']
word :  [['protest', 'VB'], 'O']
word :  [['the', 'DT'], 'O']
word :  [['war', 'NN'], 'O']
word :  [['in', 'IN'], 'O']
word :  [['Iraq', 'NNP'], 'B-geo']
word :  [['and', 'CC'], 'O']
word :  [['demand', 'VB'], 'O']
word :  [['the', 'DT'], 'O']
word :  [['withdrawal', 'NN'], 'O']
word :  [['of', 'IN'], 'O']
word :  [['British', 'JJ'], 'B-gpe']
word :  [['troops', 'NNS'], 'O']
word :  [['from', 'IN'], 'O']
word :  [['that', 'DT'], 'O']
word :  [['country', 'NN'], 'O']
word :  [['.', '.'], 'O']


## Step 2 : Splitting the train-test dataset

In [49]:
total_sentences = len(doc)
split_ratio = 0.8
train_doc,test_doc = doc[:int(total_sentences*split_ratio)],doc[int(total_sentences*split_ratio):]
print ('original doc length : ',len(doc),'sentences')
print ('train split length  : ',len(train_doc),'sentences')
print ('test split length   : ',len(test_doc),'sentences')

original doc length :  62010 sentences
train split length  :  49608 sentences
test split length   :  12402 sentences


## Step 3: Feature Engineering for each word
For each word in a sentence we need to create a feature vector. I have used the bellow feature vector  
* **POS Tag** : *Obvious choice, gives me tag like Noun Phrase, Verb, etc*  
* **Preceeding Word** : *Word before my current word*  
* **Succeeding Word** : *Word just next to my current current word*  
etc.  
These are the kind of features are the ones which I will use for tagging.
As a reference, I have used [SK-Learn NER Tagging](http://eli5.readthedocs.io/en/latest/tutorials/sklearn_crfsuite.html)

In [65]:
# How to do feature engineering.
def word2feature(sentence,word_idx):
    word = sentence[word_idx][0][0];
    postag = sentence[word_idx][0][1];
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if word_idx > 0:
        word1 = sentence[word_idx-1][0][0]
        postag1 = sentence[word_idx-1][0][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if word_idx < len(sentence)-1:
        word1 = sentence[word_idx+1][0][0]
        postag1 = sentence[word_idx+1][0][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sentence2feature(sentence):
    return [word2feature(sentence,idx) for idx in range(len(sentence))]
def sentence2label(sentence):
    return [word[1] for word in sentence]


# Trying out Word to vec on single word
print ("Features for word '", doc[0][0][0][0] + "' \n in sentence '"+ ' '.join([word[0][0] for word in doc[0]]) +"'\n")
for sentence_idx,sentence in enumerate(doc):
    for word_idx,word in enumerate(sentence):
        pprint.pprint(word2feature(sentence,word_idx))
        break;
    break;
    

Features for word ' Thousands' 
 in sentence 'Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .'

{'+1:postag': 'IN',
 '+1:postag[:2]': 'IN',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:word.lower()': 'of',
 'BOS': True,
 'bias': 1.0,
 'postag': 'NNS',
 'postag[:2]': 'NN',
 'word.isdigit()': False,
 'word.istitle()': True,
 'word.lower()': 'thousands',
 'word[-3:]': 'nds'}


## Step 4: Getting features for train and test split

In [66]:
train_data =  [sentence2feature(sentence) for sentence in train_doc]
train_label =   [sentence2label(sentence) for sentence in train_doc]
test_data =   [sentence2feature(sentence) for sentence in test_doc]
test_label =    [sentence2label(sentence) for sentence in test_doc]

## Step 5: Roll out your Training aka Label Propagation

In [83]:
import sklearn_crfsuite
import eli5
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=20,
    all_possible_transitions=False,
)
crf.fit(train_data,train_label);

In [84]:
eli5.show_weights(crf, top=5)

From \ To,O,B-art,I-art,B-eve,I-eve,B-geo,I-geo,B-gpe,I-gpe,B-nat,I-nat,B-org,I-org,B-per,I-per,B-tim,I-tim
O,2.79,0.096,0.0,0.124,0.0,1.097,0.0,0.79,0.0,0.058,0.0,1.085,0.0,1.138,0.0,1.716,0.0
B-art,-0.048,0.0,0.517,0.0,0.0,-0.129,0.0,0.0,0.0,0.0,0.0,-0.013,0.0,-0.093,0.0,-0.005,0.0
I-art,-0.069,0.0,0.282,0.0,0.0,-0.159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.016,0.0,-0.01,0.0
B-eve,-0.084,0.0,0.0,0.0,0.523,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.005,0.0
I-eve,-0.063,0.0,0.0,0.0,0.26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.006,0.0
B-geo,0.567,-0.086,0.0,-0.2,0.0,0.0,4.441,0.017,0.0,0.0,0.0,-0.555,0.0,-1.467,0.0,0.67,0.0
I-geo,0.215,-0.006,0.0,0.0,0.0,0.0,1.928,-0.117,0.0,0.0,0.0,-0.192,0.0,-0.293,0.0,0.135,0.0
B-gpe,0.685,-0.064,0.0,-0.056,0.0,-0.129,0.0,0.0,0.385,0.0,0.0,0.159,0.0,0.23,0.0,-0.446,0.0
I-gpe,0.0,0.0,0.0,0.0,0.0,-0.009,0.0,0.0,0.024,0.0,0.0,0.0,0.0,-0.001,0.0,0.0,0.0
B-nat,-0.07,0.0,0.0,0.0,0.0,-0.063,0.0,0.0,0.0,0.0,0.119,0.0,0.0,-0.029,0.0,-0.021,0.0


In [104]:
#Prediction for a single sentence
dummy_sentence = test_doc[0]
for (word,original,predicted) in zip([word[0][0] for word in dummy_sentence],crf.predict_single(sentence2feature(dummy_sentence)),sentence2label(dummy_sentence)):
    print('(',word,' , ',original,' , ',predicted,')')

( Turkey  ,  B-geo  ,  B-org )
( 's  ,  O  ,  O )
( prime  ,  O  ,  O )
( minister  ,  O  ,  O )
( ,  ,  O  ,  O )
( Recep  ,  B-per  ,  B-org )
( Tayyip  ,  I-per  ,  I-org )
( Erdogan  ,  I-per  ,  I-org )
( ,  ,  O  ,  O )
( Tuesday  ,  B-tim  ,  B-tim )
( appealed  ,  O  ,  O )
( for  ,  O  ,  O )
( calm  ,  O  ,  O )
( and  ,  O  ,  O )
( national  ,  O  ,  O )
( unity  ,  O  ,  O )
( .  ,  O  ,  O )
